In [27]:
from pyspark.sql import SparkSession
spark = (
    SparkSession.builder
    .appName("NotebookBronzeGold")
    .getOrCreate()
)


In [28]:
path_gold = "/content/diabetes_acs_full.csv"

df1 = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .option("delimiter", ";")
    .csv(path_gold)
)

df1.show(20, truncate=False)



+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+---------------------+-----------------------------------------------------------------------------------------------+-------------------------+---------------------+--------------------+------------------+------------------+--------------------+---------------------+------------------+------------------+------------------+----------------------+------------------+------------------+------------------+----------------------+---------------------+------------------+-----------------------------+-------------------------+------------------------+-------------------+------------------+------------------+------------------+------------------+----------------------+--------------------------+---------------------+-------------------------+---------------------+-------------------------+------------------+------------------+------------------+------------------

In [29]:
from pyspark.sql.functions import col

df_clean = df1.filter(col("statename").rlike("^[A-Za-z ]+$"))

df_clean.show(20, truncate=False)



+--------------+---------+----------+------------------+-----------------+---------+---------+---------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------+---------------------+--------------------+---------+------------+--------------------+---------------------+----------------+-------------+------------------+----------------------+-------------+-----+------------+----------------------+---------------------+-----------+-----------------------------+-------------------------+------------------------+-------------------+------------------+--------------+------------------+--------------+----------------------+--------------------------+---------------------+-------------------------+---------------------+-------------------------+--------------+--------------+------------------+

In [30]:
from pyspark.sql.functions import col, when, trim

flag_expr = when(
    col("diabetesactionplan").isNotNull() & (trim(col("diabetesactionplan")) != ""),
    True
).otherwise(False).alias("diabetesactionplan(True/False)")

cols = df_clean.columns
if "diabetesactionplan" in cols:
    idx = cols.index("diabetesactionplan")
    df_with_flag = df_clean.select(
        *cols[:idx],
        flag_expr,
        "diabetesactionplan",
        *cols[idx+1:]
    )
else:

    df_with_flag = df_clean.withColumn("diabetesactionplan(True/False)", flag_expr)

df_with_flag.show(20, truncate=False)


+--------------+---------+----------+------------------+-----------------+---------+---------+---------------------+------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------+---------------------+--------------------+---------+------------+--------------------+---------------------+----------------+-------------+------------------+----------------------+-------------+-----+------------+----------------------+---------------------+-----------+-----------------------------+-------------------------+------------------------+-------------------+------------------+--------------+------------------+--------------+----------------------+--------------------------+---------------------+-------------------------+---------------------+-------------------------+--------------+---

In [31]:
gold_cols = [
    "stateName","stateAbbr","statepop",
    "diabetesprevalence","diabetesIncidence",
    "funding","Footnote","diabetesactionplan(True/False)","diabetesactionplan",
    "DirectMedicalCostMillions",

    "acs_B17019EST2","acs_B17021EST1","acs_B17021EST2_PCT","acs_B17021_FAM_PCT",
    "acs_B19001_25TO35_PCT","acs_B19001_LT15_PCT","acs_B19013EST1","acs_B19025EST1",
    "acs_B19202EST1","acs_B23001_16TO24","acs_B23001_25TO64","acs_B23001_ABOVE64",
    "acs_B23001_CLF","acs_B23001_UE","acs_B23001_UE_16TO24","acs_B23001_UE_16TO24_PCT",
    "acs_B23001_UE_25TO64","acs_B23001_UE_25TO64_PCT","acs_B23001_UE_ABOVE64","acs_B23001_UE_ABOVE64_PCT",
    "acs_B23001_UE_PCT","acs_B25014_CROWD","acs_B25014_CROWD_PCT",
    "acs_C24010EST1","acs_C24010_CEMR","acs_C24010_CEMR_PCT","acs_C24010_FARM","acs_C24010_FARM_PCT",
    "acs_C24010_MPRO","acs_C24010_MPRO_PCT","acs_C24010_PTM","acs_C24010_PTM_PCT",
    "acs_C24010_SALES","acs_C24010_SALES_PCT","acs_C24010_SERVICE","acs_C24010_SERVICE_PCT"
]


df_gold = df_with_flag.select(*gold_cols)

df_gold.show(20, truncate=False)



+--------------+---------+----------+------------------+-----------------+---------+---------+------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------+--------------+--------------+------------------+------------------+---------------------+-------------------+--------------+--------------+--------------+-----------------+-----------------+------------------+--------------+-------------+--------------------+------------------------+--------------------+------------------------+---------------------+-------------------------+------------------+----------------+--------------------+--------------+---------------+-------------------+---------------+-------------------+---------------+-------------------+--------------+------------------+----------------+-----------------

In [32]:
df_gold.coalesce(1) \
    .write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv("/content/gold_output_single_csv")
